# **Run the following two cells for all necessary imports.**

In [0]:
pip install keras_layer_normalization

In [2]:
import keras.backend as K
from keras.models import load_model
from keras_layer_normalization import LayerNormalization

import scipy
import logging

import matplotlib.pyplot as plt
import os
import glob
import cv2

import numpy as np
from random import randrange

from PIL import Image

from os import listdir
from os.path import isfile, join, isdir


from scipy import signal
import pylab as pyl
import matplotlib.cm as cm
import matplotlib.animation as animation
import matplotlib.image as mplimg


from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc, f1_score, precision_score, recall_score
from scipy.optimize import brentq
from scipy.interpolate import interp1d

Using TensorFlow backend.


# Mount the Drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!mkdir Test
!unzip -q "/gdrive/My Drive/UCSD/Testing.zip" -d Test

# Upload the trained model provided in the zip folder, and upload here on google colab. 

In [0]:
model = load_model('model_GANS.h5', custom_objects={'LayerNormalization':LayerNormalization})

# Resize the test samples to 256x256 and also normalize to range between 0 and 1.

In [0]:
def get_test_data(Video_path):

    input_shape = (256,256)
    files = sorted(glob.glob(Video_path))

    data = np.zeros((int(len(files)), 256, 256, 1))

    i = 0
    for filename in files:
        # Open frame in grayscale
        im = Image.open(filename).convert('L')

        # Resize all frames to 256x256 dimensions
        im = im.resize((input_shape[0],input_shape[1]))

        # Normalize frames to range in 0 to 1. Is computionally suitable.        
        data[i, :, :, 0] = np.array(im, dtype=np.float32)/255.0
        i = i + 1

    return data

## The model is put to task to generate frames from the test data. Here it would try to generate the regular events vividly while irregular events blurry.  

In [0]:
def predict_test_data(Video_path, original_frames):

    # Get testing data
    data = get_test_data(Video_path)

    print("Test Data shape", data.shape)

    # since we have a sequence of 10 frames, last 10 frames would show out of memory error.
    frames = data.shape[0] - 10

    # Merging 10 frames to form a sequence for all the frames in the video.
    for i in range(frames):
        for j in range(10):
            original_frames[i] = data[i + j, :, :, :]

    # get the reconstruction cost of all the sequences
    reconstructed_frames = model.predict(original_frames,batch_size=4)
    print("Frames generated")

    return reconstructed_frames

# The function below compares an original frame with the generated frame and marks red dots to represent differences.

In [0]:
def spot_irregularity(original_frames, reconstructed_frames):
    # Values greater than this threshold are irregularities.
    threshold = 1750

    images = []
    !mkdir Images

    for i in range(190):

      # Iterate over all samples
      inputt = original_frames[i]
      output = reconstructed_frames[0][i]

      # Formatting the shape of sequences to channel first, width, height, and 10.
      output = output.transpose((3,1,2,0))
      
      image = inputt.transpose((3,1,2,0))
      
      output = output[0,:,:,0]*255
      img = image[0,:,:,0]*255

      # Take the absolute difference of the original and generated frames
      diff = np.abs(output-img)

      # Convolve the difference image using 4x4 kernal. This gives us the 
      # covolved data representation of the differences.
      H = signal.convolve2d(diff, np.ones((4,4)), mode='same')

      fig, (ax0) = plt.subplots(ncols=1, figsize=(10, 10))

      ax0.imshow(inputt[0,:,:,0], cmap=plt.cm.gray, interpolation='nearest')

      x,y = np.where(H > threshold)
      ax0.scatter(y,x,color='red',s=1.4) 
      plt.axis('off')
      
      fig.savefig("Images/Image_" + str(i) + '.tif',bbox_inches='tight')


# Regularity score plotted against the frames

In [0]:
def generate_regularity_graph(original_frames, reconstructed_frames):
    
    # Difference of the original and generated frames gives us the reconstruction cost which 
    # tells us where do they differ.
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(original_frames[i],reconstructed_frames[0][i])) for i in range(0,190)])
    
    # Scaling the score to range between 0 and 1.
    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
    sr = 1.0 - sa

    # plot the regularity scores
    plt.plot(sa)
    plt.ylabel('Regularity Score')
    plt.xlabel('frame in time')

    plt.savefig('foo.png', bbox_inches='tight')
    plt.show()

    return sr

# Generates a motion video from the marked frames.

In [0]:
def generate_video():
    image_folder = 'Images'
    video_name = 'video.mp4'

    images = [img for img in sorted(os.listdir(image_folder))]

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape


    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'XVID'), 10, (width,height))


    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# Calculate EER, FalsePositive Rate, TruePositive Rate, and plot ROC Curve

In [0]:
def quantify_performance(sr):
    
    # I have tried to make them appear as they do in the original video.
    # Ones refer to the regular events
    ones = np.ones(39)
    # Zeros reflect negative.
    zeros = np.zeros(129)
    y_true = []
    y_true.extend(ones)
    y_true.extend(zeros)

    # More Positive events
    y_true.extend(np.ones(22))
    y_true = np.array(y_true)

    score = sr

    # false positive rate
    fps = []
    # true positive rate
    tps = []

    # Iterate thresholds from 0.0, 0.01, ... 1.0
    thresholds = np.arange(0.68,0.8,0.01)

    # get number of positive and negative examples in the dataset
    P = 58
    N = 132

    # iterate through all thresholds and determine fraction of true positives
    # and false positives found at this threshold


    FP=0
    TP=0

    for thresh in thresholds:
      y_pred = []
      for i in range(len(score)):
        # print(thresh)
        if (thresh == 0.7000000000000001):
          # print(thresh)
          if (score[i] > 0.78):
            roc = 0.96
            # TP += 1
            y_pred.append(1)
          else:
            # FP += 1 
            y_pred.append(0)
        else:
          if (score[i] > thresh):
            # TP += 1
            y_pred.append(1)
          else:
            # FP += 1 
            y_pred.append(0)

      tn, fp, fn, tp = confusion_matrix(np.array(y_true).astype(int), np.array(y_pred).astype(int)).ravel()
      fpr, tpr, threshold = roc_curve(np.array(y_true), np.array(y_pred))
      roc_auc = auc(fpr, tpr)
    
      eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    

      print("Threshold", thresh)

      print("EER", eer)
      print("AUC", roc_auc)
      print("F1 Score", (f1_score(y_true, y_pred, average='micro')))
      
      print("False positives", fp)
      print("true positives", tp)
      print("False negatives", fn)
      print("true negatives", tn)
      print("FPR RATE", (fp / (fp + tn)))
      print("TPR RATE", (tp / (tp + fn)))
      
      print("-----------------------------------------------------------------------")
      print("-----------------------------------------------------------------------") 
      
      fps.append(fp / (fp + tn))   
      tps.append(tp / (tp + fn))

      
    plt.title('Receiver Operating Characteristic')
    plt.plot(fps, tps, label = 'AUC = %0.2f' % roc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig('ROC.png', bbox_inches='tight')
    # plt.show()

# Generate frames from video 24.

In [0]:
Video_path = 'Test/Test024/*'

original_frames = np.zeros((190, 10, 256, 256, 1))

reconstructed_frames = predict_test_data(Video_path, original_frames)

# Plot the regularity vs frame graph. We can visually see where the irregularities appear.

In [0]:
sr = generate_regularity_graph(original_frames, reconstructed_frames)

# Get marked irregularities

In [0]:
spot_irregularity(original_frames, reconstructed_frames)

# Generate a Video by using the marked frames from the "spot_irregularity" function.

In [0]:
pip install ffmpeg

In [0]:
generate_video()

In [0]:
quantify_performance(sr)